In [158]:
using JuMP
using Gurobi

include("./ConstructQD.jl")

#Test Instance 1
# global arcs = [1 2; 1 5; 2 3; 2 4; 3 8; 4 8; 5 6; 5 7; 6 8; 7 8]
# global paths = [[1 2 3 8],
#                 [1 2 4 8],
#                 [1 5 6 8],
#                 [1 5 7 8]] 


#Test Instance 2
global arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
global paths = [[1 2 3 5],
                [1 2 5],
                [1 3 5],
                [1 4 5]] 

#Problem Inputs
global v = 10
global C = 2
global c = [6, 2, 5, 10, 5, 7, 3]
global b = 5

global I = [[2],[3],[5],[7],[2,7]] #feasible set of interdictions for Test Instance 2, b=5, and c-vector shown above 
global numInterdict = length(I)

#Assuming we can always monitor exactly one arc.
global arcNum = length(arcs[:,1])
global numPaths = length(paths)
global Q_init = []

#Generate the single-arc Q-matrix and D-matrix
#Q-matrix = (Deterministic) game value for each pair of single monitored arc - path outcome
#D-matrix = arc-path traversal for each pair arc-path; D_kl = 1 if arc k is on path l
Q_init, D = ConstructQD(arcs, v, C, paths)

println("\tQ_init\t\t\t\t\tD")
for i = 1:arcNum
   println(Q_init[i,:], "\t\t", D[i,:]) 
end

#Generate the Q-matrix that perceives interdiction as monitoring decision with probability 1:
global Q = [] #Array{Int64}(undef, (0,4))
for i = 1:numInterdict
    temp = copy(Q_init)
    interdiction = I[i]
    println("Interdict ", interdiction)
    for k in interdiction
        for l = 1:numPaths
            if D[k,l] == 1
                temp[:,l] = ones(arcNum).*(-C)
            end
        end
    end
    Q = push!(Q,temp)
end
#Each inner matrix corresponds to an interdiction decision
println("# Elements in Q = ", length(Q))

#Inner problem: The evader maximizes the game value given an interdiction-monitoring policy (probability) 
function zMAX(Q,y)
    global numPaths
    gurobi_env = Gurobi.Env()
    setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
    m = Model(() -> Gurobi.Optimizer(gurobi_env))
    @variable(m, z[1:numPaths] >= 0)
    @constraint(m, sum(z[l] for l=1:numPaths) == 1)
    @objective(m, Max, y'*Q*z)
    optimize!(m)
    Obj = JuMP.objective_value.(m)
    z_now = JuMP.value.(z)
    println("Inner Obj = ", Obj)
    println("z = ", z_now)
    return z_now, Obj
end


#Starting the Outer (Minimization) Problem
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
MP = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(MP, x[1:arcNum, 1:numInterdict], Bin) #Interdiction
@variable(MP, γ[1:arcNum, 1:numInterdict], Bin) #Monitoring
@variable(MP, y[1:arcNum, 1:numInterdict] >= 0) #Interdiction-Monitoring Policy (Probability Distribution)
@variable(MP, q[1:numInterdict], Bin) #Binary variable that selects 1 interdiction solution at a time
@variable(MP, θ >= -1e6)

#Budget Constraints###
#The constraint on x-variables is not really adding value to the problem at the moment
#since we have already generated all feasible interdiction solutions (i.e., array I) before hand
@constraint(MP, sum(c[i]*x[i] for i=1:arcNum) <= b) #T
@constraint(MP, sum(y[k,i] for k=1:arcNum for i = 1:numInterdict) ==1)
#####

#Only monitoring decisions in group i in I can be non-zero if q[i] is 1 
for i = 1:numInterdict
    @constraint(MP, q[i] >= sum(y[k,i] for k=1:arcNum)) 
end

#Exactly 1 interdiction strategy must be selected at a time
@constraint(MP, sum(q[i] for i=1:numInterdict) == 1)

#Looks like we don't really need this
# @constraint(MP, θ >= sum(-C*q[i] for i=1:numInterdict) )

@objective(MP, Min, θ)
global iter =0
# println(con)
global MP_Obj = -1e6
global Inner_Obj = 1e6

#While there is discrepancy between the interdictor's and the evader's estimate on the game value 
#If there is a big enough difference then we are not at equilibrium yet.
while Inner_Obj - MP_Obj >= 1e-3
    global MP_Obj 
    global Inner_Obj 
    global iter = iter+1
    optimize!(MP)
    MP_Obj = JuMP.objective_value.(MP)
    y_now = JuMP.value.(y)
    
    #Get the interdiction solution via q[i]
    i = findall(JuMP.value.(q).==1)[1]
    println("\nIter ", iter, ": Obj = ",MP_Obj)
    println("I Group = ", i, " aka Interdict ", I[i])
    y_i = y_now[:,i]
    println("y_i = ", y_i, " aka Monitor ", findall(y_i.>0))
    
    #Since all y_i where q[i] == 0, when multiplied with the Q-matrix, adds up to 0,
    #I can choose to pass only the y-variables and the portion of the Q-matrix corresponding to q[i] = 1
    #and solve for optimal z*
    z, Inner_Obj = zMAX(Q[i], y_now[:,i])
    for i = 1:numInterdict
        #Checking elements in the added constraint below
        #The constraint must have elements in all Interdiction Grps
        println("Interdict Grp ", i, " Qz = ", Q[i]*z)
    end
    #When adding back to the MP, must add for all y-variables not just those corresponding to q[i] = 1
    con = @constraint(MP, θ >= sum(y[:,i]'*Q[i]*z for i = 1:numInterdict))
#     println(con)
end


	Q_init					D
[-2.0, -2.0, 10.0, 10.0]		[1.0, 1.0, 0.0, 0.0]
[10.0, 10.0, -2.0, 10.0]		[0.0, 0.0, 1.0, 0.0]
[10.0, 10.0, 10.0, -2.0]		[0.0, 0.0, 0.0, 1.0]
[-2.0, 10.0, 10.0, 10.0]		[1.0, 0.0, 0.0, 0.0]
[10.0, -2.0, 10.0, 10.0]		[0.0, 1.0, 0.0, 0.0]
[-2.0, 10.0, -2.0, 10.0]		[1.0, 0.0, 1.0, 0.0]
[10.0, 10.0, 10.0, -2.0]		[0.0, 0.0, 0.0, 1.0]
Interdict [2]
Interdict [3]
Interdict [5]
Interdict [7]
Interdict [2, 7]
# Elements in Q = 5
Academic license - for non-commercial use only

Iter 1: Obj = -1.0e6
I Group = 1 aka Interdict [2]
y_i = [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0] aka Monitor [3]
Academic license - for non-commercial use only
Inner Obj = 10.0
z = [1.0, 0.0, 0.0, 0.0]
Interdict Grp 1 Qz = [-2.0, 10.0, 10.0, -2.0, 10.0, -2.0, 10.0]
Interdict Grp 2 Qz = [-2.0, 10.0, 10.0, -2.0, 10.0, -2.0, 10.0]
Interdict Grp 3 Qz = [-2.0, 10.0, 10.0, -2.0, 10.0, -2.0, 10.0]
Interdict Grp 4 Qz = [-2.0, 10.0, 10.0, -2.0, 10.0, -2.0, 10.0]
Interdict Grp 5 Qz = [-2.0, 10.0, 10.0, -2.0, 10.0, -2.0, 10.0

In [138]:
#Testing on the iterative method for a simple game 
#aka no interdiction, just single-arc monitoring and path selection

global arcs = [1 2; 1 3; 1 4; 2 3; 2 5; 3 5; 4 5]
global paths = [[1 2 3 5],
                [1 2 5],
                [1 3 5],
                [1 4 5]] 
include("./ConstructQD.jl")
Q_init, D = ConstructQD(arcs, v, C, paths)

gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
Y = Model(() -> Gurobi.Optimizer(gurobi_env))

@variable(Y, y[1:arcNum]>=0)

@variable(Y, θ >= -1e6)

#Budget Constraints
@constraint(Y, sum(y[k] for k=1:arcNum) ==1)


@objective(Y, Min, θ)
for i = 1:arcNum
   println(Q_init[i,:]) 
end
for iter = 1:10
    global Q_init
    println("\nIter ", iter)
    optimize!(Y)
    MP_Obj = JuMP.objective_value.(Y)
    println("MP_Obj = ", MP_Obj)
    y_now = JuMP.value.(y)
    println("y = ", y_now)
    
    Z = Model(() -> Gurobi.Optimizer(gurobi_env))
    
    @variable(Z, z[1:numPaths]>=0)
    @constraint(Z, sum(z[l] for l=1:numPaths) ==1)
    a = @objective(Z, Max, y_now'*Q_init*z)
    println("\n",a)
    optimize!(Z)
    z_now = JuMP.value.(z)
    println("z = ", findall(z_now.>0))
    Qz = Q_init*z_now
    y_opt = findall(Qz .== minimum(Qz))
    println("y_opt = ", y_opt)
#     for l = 1:numPaths
        con = @constraint(Y, θ >= sum(y'*Q_init*z_now))
        println(con)
#     end
    println("Qz = ", Q_init*z_now)
    
end

Academic license - for non-commercial use only
[-2.0, -2.0, 10.0, 10.0]
[10.0, 10.0, -2.0, 10.0]
[10.0, 10.0, 10.0, -2.0]
[-2.0, 10.0, 10.0, 10.0]
[10.0, -2.0, 10.0, 10.0]
[-2.0, 10.0, -2.0, 10.0]
[10.0, 10.0, 10.0, -2.0]

Iter 1
MP_Obj = -1.0e6
y = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

-2 z[1] - 2 z[2] + 10 z[3] + 10 z[4]
z = [3]
y_opt = [2, 6]
-10 y[1] + 2 y[2] - 10 y[3] - 10 y[4] - 10 y[5] + 2 y[6] - 10 y[7] + θ >= 0.0
Qz = [10.0, -2.0, 10.0, 10.0, 10.0, -2.0, 10.0]

Iter 2
MP_Obj = -2.0
y = [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]

10 z[1] + 10 z[2] - 2 z[3] + 10 z[4]
z = [1]
y_opt = [1, 4, 6]
2 y[1] - 10 y[2] - 10 y[3] + 2 y[4] - 10 y[5] + 2 y[6] - 10 y[7] + θ >= 0.0
Qz = [-2.0, 10.0, 10.0, -2.0, 10.0, -2.0, 10.0]

Iter 3
MP_Obj = -2.0
y = [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

-2 z[1] + 10 z[2] - 2 z[3] + 10 z[4]
z = [2]
y_opt = [1, 5]
2 y[1] - 10 y[2] - 10 y[3] - 10 y[4] + 2 y[5] - 10 y[6] - 10 y[7] + θ >= 0.0
Qz = [-2.0, 10.0, 10.0, 10.0, -2.0, 10.0, 10.0]

Iter 4
MP_Obj = 4.0
y = [0.5